In [1]:
import pandas as pd
import os

In [2]:
from bokeh.io import output_file, save
from bokeh.palettes import Greys256
from bokeh.models import LinearColorMapper, BasicTicker, PrintfTickFormatter, ColorBar
from bokeh.plotting import figure

In [3]:
try:
    os.mkdir('output')
except:
    pass

In [4]:
Greys256.reverse()

In [5]:
datafiles = ['logs/' + fn for fn in [x for x in os.listdir('logs/') if '.csv' in x]]

In [6]:
for file in datafiles:
    df = pd.read_csv(file)
    
    Iterations = [str(i) for i in sorted([x for x in set(df['Iteration'].tolist())])]
    Neurons = [str(i) for i in sorted([x for x in set(df['Neuron'].tolist())])]
    
    df ['Iteration'] = df ['Iteration'].astype(str)
    df ['Neuron'] = df ['Neuron'].astype(str)
    
    mapper = LinearColorMapper(palette=Greys256, low=min(df['Spikes']), high=max(df['Spikes']))
    
    TOOLS = "hover,pan,box_zoom,reset,wheel_zoom"
    
    p = figure(title="Spiking Neurons over Training Iterations",
           x_range=Iterations[0::10], y_range=list(reversed(Neurons)),
           x_axis_location="above", plot_width=1920, plot_height=1080,
           tools=TOOLS, toolbar_location='below')

    p.grid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "5pt"
    p.axis.major_label_standoff = 0
    p.xaxis.major_label_orientation = 1
    
    p.rect(x="Iteration", y="Neuron", width=1, height=1,
           source=df,
           fill_color={'field': 'Spikes', 'transform': mapper},
           line_color=None)
    
    color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="5pt",
                         ticker=BasicTicker(desired_num_ticks=len(Greys256)),
                         formatter=PrintfTickFormatter(format="%d%%"),
                         label_standoff=6, border_line_color=None, location=(0, 0))
    p.add_layout(color_bar, 'right')
    
    output_file(f'output/{file[5:-4]}.html')
    save(p)